# This installs the Ollama binary and the necessary Python libraries for web searching.

In [1]:
# Install zstd for extraction and the Ollama binary
!sudo apt-get update && sudo apt-get install -y zstd
!curl -fsSL https://ollama.com/install.sh | sh

# Install Python libraries for grounding and web search
!pip install langchain-ollama langchain-community duckduckgo-search ddgs

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 https://cli.github.com/packages stable InRelease [3,917 B]
Get:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,302 kB]
Get:13 https://cli.github.com/packages stab

Step 2: Start Ollama in the Background
# Since Colab cells run sequentially, we need to start the Ollama server in a background thread so it doesn't block the rest of your code.

In [2]:
import threading
import subprocess
import time

def run_ollama_serve():
    # Launches the Ollama engine in the background
    subprocess.Popen(["ollama", "serve"])

# Start the server thread
thread = threading.Thread(target=run_ollama_serve)
thread.start()

# Wait 10 seconds for the server to initialize
time.sleep(10)

# Pull the specialized Translation model (4B is best for Colab Free)
!ollama pull translategemma:4b

# The Grounding & Translation Logic
# This is the core of your project. It searches the web for live data and then uses TranslateGemma to provide a "grounded" answer in Kannada.

In [3]:
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_ollama import OllamaLLM

# 1. Initialize tools
search = DuckDuckGoSearchRun()
# We use the translategemma model we just pulled
llm = OllamaLLM(model="translategemma:4b")

def grounded_kannada_assistant(query):
    # Get current date for context
    current_date = "January 18, 2026"
    print(f"🌐 Searching for: {query}...")

    raw_results = search.run(query)

    # Improved Prompt
    prompt = f"""
    Context: Today is {current_date}.
    Search Results: {raw_results}

    Task: Based on the search results, provide the answer in Kannada (ಕನ್ನಡ).
    If there is no news from today, summarize the most recent events found in the search results.

    KANNADA ANSWER:
    """

    response = llm.invoke(prompt)
    return response

# --- RUN A TEST ---
user_input = "what is the sbi stock price today"
print("\n--- RESULT ---")
print(grounded_kannada_assistant(user_input))


--- RESULT ---
🌐 Searching for: what is the sbi stock price today...
ಇಂದು, 18 ಜನವರಿ, 2026 ರಂದು, SBI (ಸ್ಟೇಟ್ ಬ್ಯಾಂಕ್ ಆಫ್ ಇಂಡಿಯಾ) ಬ್ಯಾಂಕ್ - ಪಬ್ಲಿಕ್ ಸೆಕ್ಟರ್ ಉದ್ಯಮದ ಶೇಖಾ ₹ 971.90 ನಲ್ಲಿ, ಇದು ಹಿಂದಿನ ಮುಕ್ತಾಯದ ಬೆಲೆಗೆ -0.25% ಕಡಿಮೆ ಆಗಿದೆ. SBI ಯ ಶೇಖಾ, State Bank of India (SBI) ಯ ಶೇಖಾ, Motilal Oswal ಅವರು ಹೇಳಿದಂತೆ, ದೇಶದ ಅತಿ ದೊಡ್ಡ ಪಬ್ಲಿಕ್ ಸೆಕ್ಟರ್ ಬ್ಯಾಂಕ್, Q1 ಫಲಿತಾಂಶಗಳನ್ನು ಘೋಷಿಸಿದೆ. ಈ ಶೇಖಾ, ಉತ್ತಮ, ಸ್ಥಿರವಾದ ಲಾಭದ ಬೆಳವಣಿಗೆ, ಸುಧಾರಿತ ಆರ್ಥಿಕ ಸ್ಥಿತಿ ಮತ್ತು ಕಾರ್ಯನಿರ್ವಹಣೆಯನ್ನು ಬಲಪಡಿಸುವ ಪ್ರಯತ್ನಗಳಿಗೆ ಅನುಗುಣವಾಗಿದೆ. Consolidated Value Rank, 61% ಇತರ ಸ್ಟಾಕ್ಸ್‌ಗಳಿಗೆ ಹೋಲಿಸಿದರೆ, ಅತ್ಯುತ್ತಮ ಶ್ರೇಣಿಯನ್ನು ಹೊಂದಿದೆ.



In [4]:
import gradio as gr
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_ollama import OllamaLLM

# 1. Setup our Grounding Logic
search = DuckDuckGoSearchRun()
llm = OllamaLLM(model="translategemma:4b")

def respond(message, history):
    """
    Gradio call function.
    'message' is the user string.
    'history' is the list of previous messages.
    """
    current_date = "January 18, 2026"

    # Step A: Perform the Search
    try:
        raw_results = search.run(message)
    except Exception as e:
        raw_results = "No search results found due to a connection error."

    # Step B: Construct the Prompt
    prompt = f"""
    Context: Today is {current_date}.
    Search Results: {raw_results}

    Task: Based on the search results, provide an update in Kannada (ಕನ್ನಡ).
    If there is no news from today, summarize the most recent events found in the text.
    Be helpful and professional.

    KANNADA ANSWER:
    """

    # Step C: Get Answer from local TranslateGemma
    response = llm.invoke(prompt)

    return response

# 2. Create the Catchy UI
demo = gr.ChatInterface(
    fn=respond,
    title="ಕನ್ನಡ AI Assistant ✍️",
    description="I search the web in real-time and answer your questions in **Kannada** using TranslateGemma.",
    examples=[
        "What is the latest update on Shah Rukh Khan?",
        "Who won the cricket match today?",
        "Current weather in Bengaluru",
        "Explain the new Gemma 3 model"
    ],
    theme=gr.themes.Soft(), # A clean, modern theme
    textbox=gr.Textbox(placeholder="Ask me anything in English...", container=False, scale=7),
)

# 3. Launch with a public link
demo.launch(share=True)

/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8772631c990e455a62.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import gradio as gr
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_ollama import OllamaLLM

# 1. Initialize our tools
search = DuckDuckGoSearchRun()
llm = OllamaLLM(model="translategemma:4b")

def respond(message, history, target_lang):
    """
    message: The user's question.
    history: List of previous chat messages.
    target_lang: The language selected from the dropdown.
    """
    current_date = "January 18, 2026"
    print(f"🌐 Searching for: {message} (Target: {target_lang})")

    # Get live web data
    try:
        raw_results = search.run(message)
    except Exception:
        raw_results = "Information currently unavailable from web search."

    # Construct the Grounding Prompt based on language choice
    prompt = f"""
    Today's Date: {current_date}
    Search Results: {raw_results}

    Task: Answer the user's question based ONLY on the search results provided.
    Output Language: {target_lang}

    If you don't find the answer in the results, state that clearly in {target_lang}.

    USER QUESTION: {message}
    {target_lang.upper()} ANSWER:
    """

    # Generate the grounded response
    response = llm.invoke(prompt)
    return response

# 2. Create the Enhanced UI
demo = gr.ChatInterface(
    fn=respond,
    title="Multi-Lingual Grounded AI 🌍",
    description="I search the web and answer in your preferred language using TranslateGemma.",

    # Add the Language Selection Dropdown
    additional_inputs=[
        gr.Dropdown(
            choices=["Kannada", "Hindi", "English"],
            value="Kannada",
            label="Select Destination Language"
        )
    ],

    # Visual Styling
    theme=gr.themes.Soft(),
    examples=[
        ["What is the latest score of India vs Australia?"],
        ["How is the weather in Mumbai today?"],
        ["Shah Rukh Khan's latest movie updates"]
    ],
    cache_examples=False
)

# 3. Launch the app
demo.launch(share=True)

/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://dbc9c7df701c9a63cc.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
